<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train a custom animal detection model with TensorFlow Lite Model Maker

In this colab notebook, we'll use the [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) to train a custom object detection model to detect Animals.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with our own custom dataset reduces the amount of training data required and will shorten the training time.


## Preparation

### Install the required packages

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support
!sudo apt-get install libportaudio2
!pip install sounddevice

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 1s (72.1 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Import the required packages.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### Prepare the dataset

This dataset contains about 70 images of 2 type of model animals: a Cow and a Hen. This is an example image from the dataset.

![animals_sample.jpg](https://drive.google.com/file/d/1HoWICSs1GIuZKdzo0HqXxEZkG3Likc4U/view?usp=sharing)

We start with downloading the dataset.

In [ ]:
!unzip -q animals.zip

replace animals/train/1653474187132.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'animals/train',
    'animals/train',
    ['cow', 'hen']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'animals/validate',
    'animals/validate',
    ['cow', 'hen']
)

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite0 to train our model.

In [ ]:
spec = model_spec.get('efficientdet_lite0')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 20`, which means it will go through the training dataset 20 times.
* Set `batch_size = 4` here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
16/16 [==============================] - 67s 2s/step - det_loss: 1.7137 - cls_loss: 1.1063 - box_loss: 0.0121 - reg_l2_loss: 0.0630 - loss: 1.7767 - learning_rate: 0.0065 - gradient_norm: 1.6643 - val_det_loss: 1.5727 - val_cls_loss: 1.0486 - val_box_loss: 0.0105 - val_reg_l2_loss: 0.0630 - val_loss: 1.6356
Epoch 2/20
16/16 [==============================] - 24s 2s/step - det_loss: 1.5555 - cls_loss: 1.0076 - box_loss: 0.0110 - reg_l2_loss: 0.0630 - loss: 1.6185 - learning_rate: 0.0049 - gradient_norm: 2.2923 - val_det_loss: 1.2414 - val_cls_loss: 0.7979 - val_box_loss: 0.0089 - val_reg_l2_loss: 0.0630 - val_loss: 1.3044
Epoch 3/20
16/16 [==============================] - 27s 2s/step - det_loss: 1.2047 - cls_loss: 0.7602 - box_loss: 0.0089 - reg_l2_loss: 0.0630 - loss: 1.2677 - learning_rate: 0.0048 - gradient_norm: 2.6795 - val_det_loss: 1.1105 - val_cls_loss: 0.7254 - val_box_loss: 0.0077 - val_reg_l2_loss: 0.0630 - val_loss: 1.1735
Epoch 4/20
16/16 [======================

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the 10 images in the validation dataset.


In [ ]:
model.evaluate(val_data)

1/1 [==============================] - 9s 9s/step



{'AP': 0.7393187,
 'AP50': 1.0,
 'AP75': 1.0,
 'AP_/cow': 0.6981094,
 'AP_/hen': 0.78052807,
 'APl': 0.7393187,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.785,
 'ARm': -1.0,
 'ARmax1': 0.77,
 'ARmax10': 0.785,
 'ARmax100': 0.785,
 'ARs': -1.0}

### Step 5: Export as a TensorFlow Lite model.



In [ ]:
model.export(export_dir='.', tflite_filename='animals.tflite')

### Step 6:  Evaluate the TensorFlow Lite model.

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore we'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite('animals.tflite', val_data)

12/12 [==============================] - 31s 2s/step



{'AP': 0.75148517,
 'AP50': 1.0,
 'AP75': 1.0,
 'AP_/cow': 0.7029703,
 'AP_/hen': 0.8,
 'APl': 0.75148517,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.775,
 'ARm': -1.0,
 'ARmax1': 0.775,
 'ARmax10': 0.775,
 'ARmax100': 0.775,
 'ARs': -1.0}

In [ ]:
# Download the TFLite model to the local computer.
from google.colab import files
files.download('animals.tflite')

FileNotFoundError: ignored